In [7]:
import numpy as np
import pandas as pd

desc = pd.read_csv('desc__mini_prev__outputs.csv', header=None)
image = pd.read_csv('image__mini_prev__outputs.csv', header=None)
multimodal = pd.read_csv('multimodal__mini_prev__outputs.csv', header=None)

In [75]:
from utils import Vocab, ValueSet

with open('data/mini-prev/attr.txt') as f:
    attr_vocab = Vocab.load(f)
    
with open('data/mini-prev/value.txt') as f:
    value_vocab = ValueSet.load(f)
    
def mcv(attr_id):
    attr = attr_vocab.id2word(attr_id)
    mcv, _ = value_vocab.partial_vocabs[attr]._counts.most_common(1)[0]
    mcv_id = value_vocab.global_vocab.word2id(mcv)
    return mcv_id

In [109]:
colnames = ['score_%i' % i for i in range(5)] + ['pred_%i' % i for i in range(5)]
d_scores = desc.iloc[:,4:].as_matrix()
d_top_5_scores = np.sort(d_scores, axis=1)[:,:5]
d_top_5_scores = pd.DataFrame(d_top_5_scores)
d_top_5_preds = np.argsort(d_scores, axis=1)[:,:5]
d_top_5_preds = pd.DataFrame(d_top_5_preds)
d_preds = pd.concat([d_top_5_scores, d_top_5_preds], axis=1)
d_preds.columns = ['d_' + i for i in colnames]

i_scores = image.iloc[:,4:].as_matrix()
i_top_5_scores = np.sort(i_scores, axis=1)[:,:5]
i_top_5_scores = pd.DataFrame(i_top_5_scores)
i_top_5_preds = np.argsort(i_scores, axis=1)[:,:5]
i_top_5_preds = pd.DataFrame(i_top_5_preds)
i_preds = pd.concat([i_top_5_scores, i_top_5_preds], axis=1)
i_preds.columns = ['i_' + i for i in colnames]

m_scores = multimodal.iloc[:,4:].as_matrix()
m_top_5_scores = np.sort(m_scores, axis=1)[:,:5]
m_top_5_scores = pd.DataFrame(m_top_5_scores)
m_top_5_preds = np.argsort(m_scores, axis=1)[:,:5]
m_top_5_preds = pd.DataFrame(m_top_5_preds)
m_preds = pd.concat([m_top_5_scores, m_top_5_preds], axis=1)
m_preds.columns = ['m_' + i for i in colnames]

In [132]:
colnames = ['uri', 'attr_id', 'correct_value_id', 'rank']
d_results = desc.iloc[:,:4]
d_results.columns = ['d_' + name if name != 'uri' else name for name in colnames]
d_results = pd.concat([d_results, d_preds], axis=1)

i_results = image.iloc[:,:4]
i_results.columns = ['i_' + name if name != 'uri' else name for name in colnames]
i_results = pd.concat([i_results, i_preds], axis=1)

m_results = multimodal.iloc[:,:4]
m_results.columns = ['m_' + name if name != 'uri' else name for name in colnames]
m_results = pd.concat([m_results, m_preds], axis=1)

combined = pd.merge(d_results, i_results)
combined = pd.merge(combined, m_results)

In [133]:
combined['mcv_id'] = combined['d_attr_id'].apply(mcv)
combined['attr'] = combined['d_attr_id'].apply(lambda x: attr_vocab.id2word(x))

In [193]:
multi_correct = combined[[a and b for a,b in zip(combined.m_pred_0!=combined.d_correct_value_id, combined.d_pred_0!=combined.d_correct_value_id)]]
multi_correct = multi_correct.drop(['i_attr_id', 'i_correct_value_id', 'm_attr_id', 'm_correct_value_id'], axis=1)
multi_correct = multi_correct.drop_duplicates()

In [213]:
i=51
results = multi_correct[['uri', 'attr', 'd_correct_value_id', 'mcv_id', 'd_pred_0', 'i_pred_0', 'm_pred_0']].iloc[25*i:25*(i+1),:]
results['d_correct_value_id'] = results['d_correct_value_id'].apply(lambda x: value_vocab.global_vocab.id2word(x))
results['mcv_id'] = results['mcv_id'].apply(lambda x: value_vocab.global_vocab.id2word(x))
results['d_pred_0'] = results['d_pred_0'].apply(lambda x: value_vocab.global_vocab.id2word(x))
results['i_pred_0'] = results['i_pred_0'].apply(lambda x: value_vocab.global_vocab.id2word(x))
results['m_pred_0'] = results['m_pred_0'].apply(lambda x: value_vocab.global_vocab.id2word(x))
results

,uri,attr,d_correct_value_id,mcv_id,d_pred_0,i_pred_0,m_pred_0
4890,product|4|116639287,year,2017,2016,2016,2016,2016
4891,product|4|116639287,year,2017,2016,2016,2016,2016
4892,product|4|116639287,year,2017,2016,2016,2016,2016
4893,product|4|116639287,year,2017,2016,2016,2016,2016
4894,product|4|1061491135,average_customer_review,(30 customer reviews),Be the first to review this item,Be the first to review this item,Be the first to review this item,Be the first to review this item
4896,product|4|530864975,certifications_and_listings,"1-UL Listed,CSA Listed",1-UL Listed,1-UL Listed,1-UL Listed,1-UL Listed
4897,product|4|-1963834732,customer_reviews,6 customer reviews 5.0 out of 5 stars,Be the first to review this product,Be the first to review this product,1 customer review 5.0 out of 5 stars,1 customer review 5.0 out of 5 stars
4898,product|4|-1580460632,customer_reviews,4 customer reviews 5.0 out of 5 stars,Be the first to review this product,1 customer review 5.0 out of 5 stars,1 customer review 5.0 out of 5 stars,1 customer review 5.0 out of 5 stars
4900,product|4|-943277618,average_customer_review,(22 customer reviews),Be the first to review this item,Be the first to review this item,Be the first to review this item,Be the first to review this item
4901,product|4|-995402389,customer_reviews,3 customer reviews 5.0 out of 5 stars,Be the first to review this product,1 customer review 5.0 out of 5 stars,1 customer review 5.0 out of 5 stars,1 customer review 5.0 out of 5 stars


In [214]:
with open('data/mapped/value.txt') as f:
    full_value_vocab = ValueSet.load(f)

['yes',
 'no',
 'black',
 'white',
 'silver',
 'gray',
 'blue',
 'stainless steel',
 'not enabled',
 'automatic',
 'get price',
 'n/a',
 'red',
 'ebay us',
 'round',
 'paperback',
 'brown',
 'seats5',
 'chrome',
 'enabled',
 'standard',
 'manual',
 '1',
 'amazon digital services llc',
 'led',
 '',
 'us',
 'clear',
 'doors',
 'not applicable',
 'gasoline',
 'pc',
 ', 30-day returns',
 'green',
 'diesel',
 '4',
 'contemporary',
 'steel',
 'petrol',
 'yellow',
 'solid',
 'false',
 'calculated at checkout',
 'medium',
 'in stock',
 'none',
 'grey',
 'us & many other countries',
 'traditional',
 'rohs',
 'metal',
 'multicolor',
 'residential',
 'lcd',
 'front wheel drive',
 'indoor',
 'emissions',
 'worldwide',
 'rectangular',
 'bronze',
 'seats',
 'other',
 'vehicle height-, vehicle width',
 '5',
 '2',
 'plastic',
 'calculated in cart',
 'wired',
 'wood',
 'gold',
 'rectangle',
 '3',
 'external',
 'usb',
 '16:9',
 'area rug',
 'baby product',
 'some assembly required',
 'quartz',
 '2 front